# マルコフモデル

*Markov Model*

マルコフ過程に従う確率モデル。マルコフ連鎖とも。  
マルコフ過程とは、ある状態から次の状態へ遷移する確率が、その状態のみに依存する確率過程のこと。

In [1]:
import random


---

## 文章生成

マルコフモデルを用いて文章を生成する。状態を単語とし、ある単語の次に続く単語の確率を予測する。

学習データとして、いくつかの単語列（=文章）を用意する。学習データに出現する全ての単語を対象に、その次に続く単語とその確率を記録することで、マルコフモデルが完成する。確率は単語の出現頻度から定める。

例として以下の単語列を考えてみよう。

- 今日 は いい 天気 です
- 私 は 今日 カレー を 食べ ました
- 私 は カレー が 好き です

この3つの単語列を学習させてマルコフモデルを作成する。

まず、1つ目に選ばれる可能性のある単語として「今日」と「私」がある。どちらが選ばれるかはランダムであるが、「今日」は1回、「私」は2回出現しているので、「今日」が選ばれる確率は1/3、「私」が選ばれる確率は2/3と定義する。  
次に、この次に選ばれる可能性のある単語と確率を定義する。「今日」の次に来る可能性がある単語は「は」と「カレー」で、出現頻度は同じであるため確率は一様となる。  
この流れで全ての単語について次に続く単語と確率を定義するとこうなる。 

In [2]:
data = [
    "今日 は いい 天気 です",
    "私 は 今日 カレー を 食べ ました",
    "私 は カレー が 好き です"
]

words = {}
for sent in data:
    sent = sent.split()
    for w1, w2 in zip(sent[:-1], sent[1:]):
        if w1 not in words:
            words[w1] = {}
        if w2 not in words[w1]:
            words[w1][w2] = 0
        words[w1][w2] += 1

for w1 in words:
    total = sum(words[w1].values())
    for w2 in words[w1]:
        words[w1][w2] /= total

words

{'今日': {'は': 0.5, 'カレー': 0.5},
 'は': {'いい': 0.3333333333333333,
  '今日': 0.3333333333333333,
  'カレー': 0.3333333333333333},
 'いい': {'天気': 1.0},
 '天気': {'です': 1.0},
 '私': {'は': 1.0},
 'カレー': {'を': 0.5, 'が': 0.5},
 'を': {'食べ': 1.0},
 '食べ': {'ました': 1.0},
 'が': {'好き': 1.0},
 '好き': {'です': 1.0}}

「今日」の次は「は」と「カレー」が$\frac{1}{2}$ずつ、「は」の次は「いい」「今日」「カレー」が$\frac{1}{3}$ずつという風に確率を定義することが出来た。

では、この確率を用いて文章を生成してみる。

In [3]:
def generate(words, start_words):
    word, = random.choices(list(start_words), weights=start_words.values())
    sentence = []
    while word in words:
        next_words = words[word]
        word, = random.choices(list(next_words), weights=next_words.values())
        sentence.append(word)
    return ' '.join(sentence)

for _ in range(10):
    sentence = generate(words, {"今日": 1/3, "私": 2/3})
    print(sentence)

は いい 天気 です
は 今日 は カレー を 食べ ました
は 今日 は 今日 は いい 天気 です
カレー が 好き です
カレー を 食べ ました
カレー が 好き です
は 今日 カレー が 好き です
は 今日 カレー が 好き です
カレー を 食べ ました
は いい 天気 です


自然な文章もあれば、意味不明な文章もあるだろう。単語を予測する際に直前の単語のみを参照しているため、自然な文章を生成することは難しい。

### markovify

マルコフモデルを用いて文章を生成するためのライブラリ。

- [jsvine/markovify: A simple, extensible Markov chain generator.](https://github.com/jsvine/markovify)

これを使うと簡単にマルコフモデルを用いた言語モデルを実装できる。

In [4]:
import markovify

In [5]:
data = [
    "今日 は いい 天気 です",
    "私 は 今日 カレー を 食べ ました",
    "私 は カレー が 好き です"
]
model = markovify.Text(data, state_size=1)

これで学習が完了した。文章を生成してみる。

In [6]:
for _ in range(10):
    sentence = model.make_sentence()
    print(sentence)

私 は いい 天気 です
私 は いい 天気 です
私 は 今日 は カレー が 好き です
私 は カレー を 食べ ました
私 は いい 天気 です
私 は カレー を 食べ ました
今日 は 今日 は 今日 カレー を 食べ ました
私 は いい 天気 です
私 は カレー を 食べ ました
私 は 今日 は カレー を 食べ ました
